Importing the required libraries

In [1]:
import os
import json
import trimesh
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

Reading the category labels from the saved json file

In [2]:
# Loading the ShapeNetCore labels
with open('./labels.json', 'r') as json_file:
    labels = json.load(json_file)

Setting the device

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Creating a ShapeNetDataset class for sampling points from meshes

In [4]:
class ShapeNetDataset(Dataset):
    def __init__(self, file_paths, labels):
        self.file_paths = file_paths
        self.labels = labels

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        mesh = trimesh.load(path, force='mesh')
        points = mesh.sample(1024)  # Sample 1024 points from the mesh
        points = torch.tensor(points, dtype=torch.float32)
        label = self.labels[idx]
        return points, label

Creating and loading the datasets as training, validation, and testing datasets

In [5]:
def load_data(root_dir):
    file_paths = []
    labels = []
    label_map = {}
    for idx, directory in enumerate(sorted(os.listdir(root_dir))):
        dir_path = os.path.join(root_dir, directory)
        if os.path.isdir(dir_path):
            label_map[directory] = idx
            for models in os.listdir(dir_path):
                model_directory = os.path.join(dir_path, models, 'models/model_normalized.obj')
                if os.path.isfile(model_directory):
                    file_paths.append(model_directory)
                    labels.append(idx)
    return file_paths, labels, label_map

def create_datasets(root_dir, test_size=0.2, val_size=0.1):
    file_paths, labels, label_map = load_data(root_dir)
    # Splitting dataset into train, validation, and test sets
    train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=test_size, random_state=42)
    train_paths, val_paths, train_labels, val_labels = train_test_split(train_paths, train_labels, test_size=val_size / (1 - test_size), random_state=42)

    train_dataset = ShapeNetDataset(train_paths, train_labels)
    val_dataset = ShapeNetDataset(val_paths, val_labels)
    test_dataset = ShapeNetDataset(test_paths, test_labels)
    return train_dataset, val_dataset, test_dataset, label_map

Creating the dataloaders

In [6]:
root_dir = 'C:/ShapeNetCore'  # ShapeNetCore directory
# root_dir = 'C:/Users/rohan/OneDrive/Desktop/ShapeNet'
train_dataset, val_dataset, test_dataset, label_map = create_datasets(root_dir)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

Saving and loading model checkpoints

In [7]:
def save_checkpoint(state, filename):
    """Save a model checkpoint."""
    os.makedirs('./checkpoints', exist_ok=True)
    torch.save(state, os.path.join('./checkpoints', filename))

def load_checkpoint(filepath, model, optimizer=None):
    """Load model checkpoint."""
    if not os.path.isfile(filepath):
        print(f"No checkpoint found at '{filepath}'")
        return None

    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer'])

    print(f"Loaded checkpoint '{filepath}' (epoch {checkpoint['epoch']})")
    return checkpoint

Training and Testing functions for the model

In [8]:
def train_model(model, train_loader, val_loader, num_epochs=25, save_path='model_checkpoint.pth'):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for points, labels in train_loader:
            points, labels = points.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(points)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        # Validation
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for points, labels in val_loader:
                points, labels = points.to(device), labels.to(device)
                outputs = model(points)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()

        average_val_loss = total_val_loss / len(val_loader)

        # Print training and validation loss
        print(f"Epoch {epoch+1}, Train Loss: {total_train_loss/len(train_loader)}, Validation Loss: {average_val_loss}")
        
        # Checking updates in validation loss
        if average_val_loss < best_val_loss:
            print(f"Validation loss decreased ({best_val_loss} --> {average_val_loss}). Saving checkpoint")
            best_val_loss = average_val_loss
            # Save checkpoint
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_val_loss': best_val_loss,
                'optimizer': optimizer.state_dict(),
            }, filename=save_path)

def test_model(model, test_loader):
    model.eval()
    total_test_loss = 0
    pred_labels = []
    actual_labels = []
    criterion = torch.nn.CrossEntropyLoss()

    with torch.no_grad():
        for points, labels in test_loader:
            points, labels = points.to(device), labels.to(device)
            outputs = model(points)
            loss = criterion(outputs, labels)
            total_test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            pred_labels.extend(predicted.cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())

    print('Test Scores:-')
    print(f"Accuracy: {accuracy_score(actual_labels, pred_labels)}")
    print(f"Precision: {precision_score(actual_labels, pred_labels, average='macro', zero_division=1)}")
    print(f"Recall: {recall_score(actual_labels, pred_labels, average='macro')}")
    print(f"F1 Score: {f1_score(actual_labels, pred_labels, average='macro')}")
    print("Confusion Matrix:\n", confusion_matrix(actual_labels, pred_labels))
    print(f"Test Loss: {total_test_loss/len(test_loader)}")

Defining the Neural Network

In [9]:
class ThreeDClassifier(nn.Module):
    def __init__(self, num_classes, dropout_prob=0.3):
        super().__init__()
        self.fc1 = nn.Linear(1024 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)  # Applies LogSoftmax on each output

    def forward(self, x):
        x = x.reshape(x.size(0), -1)  # Flatten the points
        x = self.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout after first ReLU activation
        x = self.relu(self.fc2(x))
        x = self.dropout(x)  # Apply dropout after second ReLU activation
        x = self.fc3(x)
        x = self.log_softmax(x)  # Apply LogSoftmax after the final linear layer
        return x

Training the model

In [10]:
model = ThreeDClassifier(num_classes=len(label_map)).to(device)
train_model(model, train_loader, val_loader, num_epochs=25)

Epoch 1, Train Loss: 2.8292373881299784, Validation Loss: 2.571763321331569
Validation loss decreased (inf --> 2.571763321331569). Saving checkpoint
Epoch 2, Train Loss: 2.5502274804306135, Validation Loss: 2.3479396633874803
Validation loss decreased (2.571763321331569 --> 2.3479396633874803). Saving checkpoint
Epoch 3, Train Loss: 2.3929658241827614, Validation Loss: 2.18578534693945
Validation loss decreased (2.3479396633874803 --> 2.18578534693945). Saving checkpoint
Epoch 4, Train Loss: 2.3074664202529664, Validation Loss: 2.1837177371978758
Validation loss decreased (2.18578534693945 --> 2.1837177371978758). Saving checkpoint
Epoch 5, Train Loss: 2.2468539809064194, Validation Loss: 2.072497758751824
Validation loss decreased (2.1837177371978758 --> 2.072497758751824). Saving checkpoint
Epoch 6, Train Loss: 2.2045241302863605, Validation Loss: 2.013725819360642
Validation loss decreased (2.072497758751824 --> 2.013725819360642). Saving checkpoint
Epoch 7, Train Loss: 2.1622362014

Testing the model

In [11]:
test_model(model, test_loader)

Test Scores:-
Accuracy: 0.4782277274892806
Precision: 0.7220546794631302
Recall: 0.13433136439739737
F1 Score: 0.13228130375770727
Confusion Matrix:
 [[710   0   0 ...   0  20   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   9   2   0]
 [ 17   0   0 ...   5 110   0]
 [  0   0   0 ...   0   0   0]]
Test Loss: 1.793131581175895


Predicting the ShapeNet class label for a 3D object

In [12]:
def predict_3d_model(model, file_path, label_map):
    """
    Predict the class of a 3D model given its file path.
    
    Args:
    - model: Trained PyTorch model for prediction.
    - file_path: Path to the .obj file to be predicted.
    - label_map: Dictionary mapping class indices back to labels.
    
    Returns:
    - label: Predicted class label of the 3D model.
    """
    model.eval()  # Set model to evaluation mode
    try:
        # Load and preprocess the mesh
        mesh = trimesh.load(file_path, force='mesh')
        points = mesh.sample(1024)  # Sample points as done in training
        points = torch.tensor(points, dtype=torch.float32)
        points = points.unsqueeze(0).to(device)  # Add batch dimension
        
        # Predict using the model
        with torch.no_grad():
            outputs = model(points)
            _, predicted_idx = torch.max(outputs, 1)
            predicted_label = list(label_map.keys())[list(label_map.values()).index(predicted_idx.item())]

        return predicted_label
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

Loading from a saved checkpoint and predicting classes

In [21]:
# Loading the trained model
model = ThreeDClassifier(num_classes=len(label_map))
model.to(device)
optimizer = torch.optim.Adam(model.parameters())

# Loading the checkpoint
checkpoint = load_checkpoint('./checkpoints/model_checkpoint.pth', model, optimizer)
if checkpoint:
    print("Successfully loaded model and optimizer states")

# Path to the 3D model whose class is to be predicted
file_path = "C:/ShapeNetCore/02747177/ffe5f0ef45769204cb2a965e75be701c/models/model_normalized.obj"

# Predicting the label
predicted_label = predict_3d_model(model, file_path, label_map)
print(f"The predicted class label for the model is: {labels[predicted_label]}")

Loaded checkpoint './checkpoints/model_checkpoint.pth' (epoch 24)
Successfully loaded model and optimizer states
The predicted class label for the model is: chair


Saving the entire model

In [22]:
os.makedirs('./models', exist_ok=True)
model.to('cpu')
torch.save(model, './models/entire_model.pth')

In [ ]:
# TODO
# Improve model accuracy and other metrics
# Issue: Model predicts 'chair' class for almost all items